In [1]:
#importing libraries 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sklearn
import math

In [2]:
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.metrics import f1_score as f1
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression as logreg
from sklearn.linear_model import LinearRegression as linreg
from sklearn.metrics import mean_squared_error as rmse

from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score ,roc_auc_score as auc, accuracy_score as acc, confusion_matrix, roc_curve, precision_score, recall_score, precision_recall_curve,average_precision_score,average_precision_score as aps,log_loss as ll

from pandas.core.common import SettingWithCopyWarning
from sklearn.exceptions import FitFailedWarning
import warnings
warnings.simplefilter(action='ignore', category=FitFailedWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [3]:
from sklearn.tree import DecisionTreeRegressor as dtr,export_graphviz
from sklearn.model_selection import GridSearchCV as gs, RandomizedSearchCV as rs
from sklearn.metrics import make_scorer
import graphviz

## random state

In [4]:
r=6

In [5]:
data = pd.read_csv('spotify_data.csv')
data.head()

,Position,Track Name,Artist,Streams,URL,Date,Region
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,2017-01-01,ec
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,2017-01-01,ec
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,2017-01-01,ec
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,2017-01-01,ec
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,2017-01-01,ec


## Missing Value

In [6]:
data.isnull().sum()

Position        0
Track Name    657
Artist        657
Streams         0
URL             8
Date            0
Region          0
dtype: int64

In [7]:
data.dropna(inplace=True)

## Preprocessing

### track_names

In [8]:
data.head()

,Position,Track Name,Artist,Streams,URL,Date,Region
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,2017-01-01,ec
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,2017-01-01,ec
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,2017-01-01,ec
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,2017-01-01,ec
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,2017-01-01,ec


In [9]:
le = preprocessing.LabelEncoder()
data['Track Name']=le.fit_transform(data['Track Name'])
data['Artist']=le.fit_transform(data['Artist'])
data['Region']=le.fit_transform(data['Region'])

removing url column

In [10]:
data.drop(['URL'], axis=1,inplace=True)

In [11]:
data.head()

,Position,Track Name,Artist,Streams,Date,Region
0,1,12081,897,19272,2017-01-01,15
1,2,2412,5194,19270,2017-01-01,15
2,3,10924,6331,15761,2017-01-01,15
3,4,15445,4854,14954,2017-01-01,15
4,5,12461,2523,14269,2017-01-01,15


## Date

In [12]:
data['Date'].max(), data['Date'].min()

('2018-01-09', '2017-01-01')

In [13]:
data['Date'] = pd.to_datetime(data['Date'], format= '%Y-%m-%d')
data.dtypes

Position               int64
Track Name             int64
Artist                 int64
Streams                int64
Date          datetime64[ns]
Region                 int64
dtype: object

In [14]:
data['Date'].max(), data['Date'].min()

(Timestamp('2018-01-09 00:00:00'), Timestamp('2017-01-01 00:00:00'))

### extracting

year

In [15]:
data['Date'].dt.year

0          2017
1          2017
2          2017
3          2017
4          2017
           ... 
3441192    2018
3441193    2018
3441194    2018
3441195    2018
3441196    2018
Name: Date, Length: 3440540, dtype: int64

In [16]:
data['Date'].dt.year.value_counts()

2017    3355829
2018      84711
Name: Date, dtype: int64

month

In [17]:
data['Date'].dt.month

0          1
1          1
2          1
3          1
4          1
          ..
3441192    1
3441193    1
3441194    1
3441195    1
3441196    1
Name: Date, Length: 3440540, dtype: int64

In [18]:
data['Date'].dt.month.value_counts()

1     361156
12    297611
7     290305
10    289677
8     289439
3     285994
11    280138
9     280084
4     274391
6     269533
5     268020
2     254192
Name: Date, dtype: int64

day of month

In [19]:
data['Date'].dt.day

0          1
1          1
2          1
3          1
4          1
          ..
3441192    9
3441193    9
3441194    9
3441195    9
3441196    9
Name: Date, Length: 3440540, dtype: int64

In [20]:
data['Date'].dt.day.value_counts()[0:5]

8    120677
9    120677
7    120522
5    120435
4    120404
Name: Date, dtype: int64

In [21]:
len(data['Date'].dt.day.value_counts())

31

day of year

In [22]:
data['Date'].dt.dayofyear

0          1
1          1
2          1
3          1
4          1
          ..
3441192    9
3441193    9
3441194    9
3441195    9
3441196    9
Name: Date, Length: 3440540, dtype: int64

In [23]:
data['Date'].dt.dayofyear.value_counts()

5     18432
6     18402
9     18389
8     18220
4     18197
      ...  
23     8891
29     8889
11     8875
10     8840
15     8829
Name: Date, Length: 362, dtype: int64

In [24]:
len(data['Date'].dt.dayofyear.value_counts())

362

day of week

In [25]:
data['Date'].dt.dayofweek

0          6
1          6
2          6
3          6
4          6
          ..
3441192    1
3441193    1
3441194    1
3441195    1
3441196    1
Name: Date, Length: 3440540, dtype: int64

In [26]:
data['Date'].dt.dayofweek.value_counts()

0    497277
5    493745
3    493319
1    491527
6    491422
4    490420
2    482830
Name: Date, dtype: int64

day of week name

In [27]:
data['Date'].dt.day_name()

0           Sunday
1           Sunday
2           Sunday
3           Sunday
4           Sunday
            ...   
3441192    Tuesday
3441193    Tuesday
3441194    Tuesday
3441195    Tuesday
3441196    Tuesday
Name: Date, Length: 3440540, dtype: object

## Put it all together

In [28]:
date_data = pd.DataFrame({"Year": data['Date'].dt.year,
              "Month": data['Date'].dt.month,
              "Day Of Month": data['Date'].dt.day,
              "Day Of Year": data['Date'].dt.dayofyear,
              "Day Of Week": data['Date'].dt.dayofweek,
              "Day Of Week Name": data['Date'].dt.day_name()
             })
date_data.head()

,Year,Month,Day Of Month,Day Of Year,Day Of Week,Day Of Week Name
0,2017,1,1,1,6,Sunday
1,2017,1,1,1,6,Sunday
2,2017,1,1,1,6,Sunday
3,2017,1,1,1,6,Sunday
4,2017,1,1,1,6,Sunday


In [29]:
data.shape

(3440540, 6)

In [30]:
date_data.shape

(3440540, 6)

joining date_data to main data

In [31]:
data=data.join(date_data.set_index(data.index), on=data.index).copy()

removing date column

In [32]:
data.drop(['Date'], axis=1,inplace=True)

In [33]:
data.head()

,Position,Track Name,Artist,Streams,Region,Year,Month,Day Of Month,Day Of Year,Day Of Week,Day Of Week Name
0,1,12081,897,19272,15,2017,1,1,1,6,Sunday
1,2,2412,5194,19270,15,2017,1,1,1,6,Sunday
2,3,10924,6331,15761,15,2017,1,1,1,6,Sunday
3,4,15445,4854,14954,15,2017,1,1,1,6,Sunday
4,5,12461,2523,14269,15,2017,1,1,1,6,Sunday


we will need Day Of Week Name for eda but we dont need for forecasting so temporarily removing it

In [34]:
data.drop(['Day Of Week Name'], axis=1,inplace=True)
data.head()

,Position,Track Name,Artist,Streams,Region,Year,Month,Day Of Month,Day Of Year,Day Of Week
0,1,12081,897,19272,15,2017,1,1,1,6
1,2,2412,5194,19270,15,2017,1,1,1,6
2,3,10924,6331,15761,15,2017,1,1,1,6
3,4,15445,4854,14954,15,2017,1,1,1,6
4,5,12461,2523,14269,15,2017,1,1,1,6


In [35]:
data.shape

(3440540, 10)

basic analysis

positions are rankings of tracks from 1 to 200

In [36]:
data['Position'].value_counts()

1      19667
7      19660
6      19659
5      19655
8      19654
       ...  
196    14448
197    14434
198    14421
199    14411
200    14402
Name: Position, Length: 200, dtype: int64

In [37]:
len(data['Position'].value_counts())

200

postion 1 

In [38]:
data[data['Position']==1]['Track Name'].value_counts()

12905    3639
3342     2342
16725    1690
3341     1560
9310     1180
         ... 
16609       1
12178       1
15628       1
7677        1
8226        1
Name: Track Name, Length: 256, dtype: int64

on postion 1 we have 256 tracks for example

In [39]:
len(data[data['Position']==1]['Track Name'].value_counts())

256

position 200

In [40]:
data[data['Position']==200]['Track Name'].value_counts()

16216    75
8206     62
13641    60
8672     55
258      50
         ..
13626     1
12563     1
5561      1
9514      1
18438     1
Name: Track Name, Length: 4151, dtype: int64

 on postion 200 we have 4151 tracks for example

In [41]:
len(data[data['Position']==200]['Track Name'].value_counts())

4151

track name and artist

In [42]:
data['Track Name'].value_counts()

12905    19365
1538     16775
3341     16284
13414    16194
12235    16177
         ...  
13929        1
9759         1
11631        1
15954        1
18135        1
Name: Track Name, Length: 18597, dtype: int64

In [43]:
len(data['Track Name'].value_counts())

18597

In [44]:
data[data['Track Name']==1]['Artist']

305774     1719
1703107    1719
1704313    1719
1704468    1719
1705703    1719
1705880    1719
1804109    1719
2141150    1719
2565583    1719
2566789    1719
2566962    1719
2568174    1719
2568373    1719
Name: Artist, dtype: int64

In [45]:
data[data['Track Name']==200]['Artist']

643779     15
643979     15
644176     15
644389     15
801485     15
           ..
3161987    15
3162193    15
3162395    15
3162597    15
3162794    15
Name: Artist, Length: 457, dtype: int64

note for each track name there is one and only one artist so each track corresponds to one artist so for prediction artist variable is not required, for eda visualization and data analysis though we will require artist name and trck name

## making model

target column is position

In [ ]:
target='Position'

train, validation and test datasets